<a href="https://colab.research.google.com/github/spiderassassin/TensionExtractor/blob/main/TensionExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tension Extractor -  Notebook
Welcome! This tool can be used to extract tension curves from Let's play videos. The algorithm uses amplitude to calculate the tension values. For more information, checkout this [video](https://www.youtube.com/watch?v=ZssSUZanbAk).

In [ ]:
#@markdown # Enter the youtube link here
Link = "" # @param {"type":"string","placeholder":"Youtube Link"}

!pip install pytubefix

from glob import glob
import numpy as np
import librosa
import librosa.display
import os
import IPython.display as ipd
import sklearn
from sklearn.decomposition import FastICA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from scipy.io import wavfile
from tqdm import tqdm
from sklearn.preprocessing import minmax_scale
import math
import scipy


from pytubefix import YouTube
from pytubefix.cli import on_progress

import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='librosa')
warnings.filterwarnings('ignore', category=FutureWarning, module='librosa')
warnings.filterwarnings('ignore', message='.*PySoundFile.*failed.*')

def moving_average(data, window_size):
    pad_width = window_size // 2
    padded_data = np.pad(data, pad_width, mode='edge')
    weights = np.repeat(1.0, window_size) / window_size
    smoothed = np.convolve(padded_data, weights, 'valid')
    return smoothed

def dataprocess(arr):
    arr = minmax_scale(arr)
    arr = np.square(np.cos(arr * math.pi))
    arr = moving_average(arr, 100)
    arr = moving_average(arr, 100)
    arr = moving_average(arr, 100)
    arr = minmax_scale(arr)

    return arr

def seconds_to_min_sec(seconds):
    minutes = int(seconds // 60)
    seconds = seconds % 60
    return f'{minutes:02}:{int(seconds)}'



try:
    # object creation using YouTube
    yt = YouTube(Link, on_progress_callback = on_progress)
except:
    #to handle exception
    print("Connection Error")

stream = yt.streams.filter(only_audio=True).first()

out_file = stream.download()

new_file = 'video.mp3'
os.rename(out_file, new_file)

y, sr = librosa.load(new_file)

# Compute RMS (Root Mean Square) for each frame
rms = librosa.feature.rms(y=y)

# Normalize RMS values to the range [0, 1]
rms_normalized = rms / np.max(rms)

rms_normalized = rms_normalized[0]





# Get the corresponding time for each frame (in seconds)
times = librosa.times_like(rms)
print(len(times))

rms_normalized = moving_average(rms_normalized, 800)
rms_normalized = moving_average(rms_normalized, 800)
rms_normalized = minmax_scale(rms_normalized)
#rms_normalized = np.square(np.cos(rms_normalized * math.pi))
#rms_normalized = moving_average(rms_normalized, 100)

#rms_normalized = dataprocess(rms_normalized)

rms_normalized = rms_normalized[:len(times)]
print(len(rms_normalized))

time_labels = [seconds_to_min_sec(t) for t in times]


# Plot normalized RMS loudness over time
plt.figure(figsize=(12, 6))
plt.plot(times, rms_normalized, label="Normalized RMS Loudness", color='royalblue', linewidth=2)

# Adding grid, labels, title
plt.xlabel("Time (minutes:seconds)", fontsize=14, weight='bold')
plt.ylabel("Tension (0 to 1)", fontsize=14, weight='bold')
plt.title("Tension over Time ", fontsize=16, weight='bold')

# Set custom x-ticks with the formatted time labels (rotate them for better visibility)
plt.xticks(ticks=times[::int(len(times) / 10)], labels=time_labels[::int(len(times) / 10)], rotation=45, fontsize=12)

# Adding a grid for readability
plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)

# Add a legend with a transparent background
plt.legend(loc="upper right", fontsize=12, frameon=False)

# Tight layout to make sure everything fits
plt.tight_layout()

# Show the plot
plt.show()

